In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Kaggle Dataset의   Trip Adviser Hotel Review Dataset

In [2]:
import os
os.environ['KAGGLE_USENAME'] = "beasterea1109"
os.environ['KAGGLE_KEY'] = "a876ec11ffb6bb37bf785a0af44506cf"

In [3]:
import kaggle

In [5]:
!kaggle datasets download -d andrewmvd/trip-advisor-hotel-reviews


  0%|          | 0.00/5.14M [00:00<?, ?B/s]
 39%|###8      | 2.00M/5.14M [00:00<00:00, 18.5MB/s]
 78%|#######7  | 4.00M/5.14M [00:00<00:00, 18.4MB/s]
100%|##########| 5.14M/5.14M [00:00<00:00, 18.7MB/s]


#### INTRO
1. dataset은 그냥 csv파일인데, hotel에 대한 review text와 함께 rating이 제공이 되어있다.
2. 이번 notebook의 목적은 review데이터를 받고 해당 데이터에 따른 rating을 분석해 주는 것이다.
3. loss함수로 MAE, 혹은 RMSE를 사용하여서 해결을 하는 것이 좋을 것이라고 생각한다.
4. dataset에 대한 pipeline과 여기서 중요한 단어에는 더 가중치를 둔다거나 하는 방법을 사용해서 해결을 할 예정이다.
    - Convolution layer + RNN
    - RNN  
    위의 두가지 모델을 모두 모델링해서 정확도가 어느 쪽이 더 높은지 확인해 볼 예정이다.
5. 무엇보다 Many to One의 형태이기 때문에 모델링을 하는데에 크게 어려움이 있을 것으로 보이지는 않는다.
6. 다만 텍스트 분석을 위해서 **전처리, tokenizing, embedding**을 잘 해 주는 것이 중요할 것이다.

In [7]:
import zipfile
data = zipfile.ZipFile('C:\\Users\\이지혜\\trip-advisor-hotel-reviews.zip')
data.extractall('C:\\Users\\이지혜\\OneDrive\\문서')

In [2]:
file_root = "C:\\Users\\이지혜\\OneDrive\\문서\\tripadvisor_hotel_reviews.csv"
hotel_data = pd.read_csv(file_root)

In [3]:
hotel_data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


### 1. Train_Test_Split + Text 추출

In [140]:
review = hotel_data['Review']

In [141]:
target = hotel_data['Rating']

In [142]:
len(target)

20491

In [143]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(review, target, test_size = 0.2)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2)

In [144]:
x_train[7]

'excellent staff, housekeeping quality hotel chocked staff make feel home, experienced exceptional service desk staff concierge door men maid service needs work, maid failed tuck sheets foot bed instance soiled sheets used, staff quickley resolved soiled sheets issue, guess relates employee not reflection rest staff.we received excellent advice concierge regarding resturants area happy hour wine tasting nice touch staff went way make feel home.great location like close good food shopping took play 5th street theather well.pikes market pioneer square access mono rail short walking distance,  '

In [44]:
len(x_train), len(y_train)

(13113, 13113)

### 2-1. Preprocessing // WordDict

**Preprocessing**
- 간단하게 불필요한 문자를 없애주는 preprocess, 즉 전처리 함수를 만들어서 사용한다.
- 사실이 전처리 함수를 사용하면 따로 padding과정을 거칠 필요는 없다.
    - 먼저 각 리뷰에서 처음 300개의 글자만 남겨서 훈련 속도를 높일 수 있도록 한다.
    - 그리고 정규식을 이용해서 문자와 작은 따옴표가 아닌 다른 문자는 모두 공백으로 바꾸고 <br\>또한 공백으로 바꾼다.
    - 마지막에는 리뷰를 공백을 기준으로 나누는데, 이렇게 하면 길이가 모두 다른 ragged tensor이 반환될 것이다.
    - 따라서 이를 밀집 텐서로 바꾸고 padding token으로 모든 리뷰를 padding 한다.  

In [136]:
def preprocess(x_data):
    x_data = tf.strings.substr(x_data, 0, 300)
    x_data = tf.strings.regex_replace(x_data, b"<br\\s*/?>", b" ")
    x_data = tf.strings.regex_replace(x_data, b"[^a-zA-Z']", b" ")
    x_data = tf.strings.split(x_data)
    return x_data.to_tensor(default_value = b"<pad>")

#### Word Dict
- 직접 단어 사전을 만들어서 이를 이용해서 word2_idx의 역할을 하는 dictionary의 자료형을 갖는 데이터를 만들 것이다.
- 뿐만 아니라 counter을 이용해서 단어의 사용 횟수를 고려하여 첫 단어부터 가장 많이 사룔된 단어들 1000개를 이용해서 index로 바꿔 준 뒤에 처리를 할 예정이다.
- 단, 이경우에는 위에서 사용한 preprocessing 함수는 굳이 사용하지 않을 예정이다.
    - tensor의 형태로 변형이 되어 있어서 사용하기가 번거롭기 때문이다.

**Word_Dict - 1. Using Counter**
- 단어의 most_common만을 사용하는 등의 방법이 가능해서, 즉 사용 횟수가 많을 수록 데이터셋에 반영하는 것이 가능하기 때문에 좋다.
1. tokened_review 라는 리스트에 모든 문장에 대해서 단어별로 tokenization을 하기 위해서 공백을 기준으로 분류를 하여서 나누어 저장을 했다.
2. Counter이라는 클래스를 이용해서 tokened_review의 모든 단어들을 리스트의 형태로 update를 했다.
3. 가장 많이 사용이 되는 10000개의 단어만을 저장하고 나머지를 삭제한 뒤에 
4. 혹시나 있을지 모르는 oov vocabulary의 개수 100개를 포함하여서 vocab_init이라는 단어 사전을 만들었다.
5. ```table.lookup()```을 이용하여 모든 tokenize된 review들을 인덱스화 하고 모든 review의 길이를 맞추기 위해서 pad_sequence를 적용하였다.
6. 마지막으로 dataset을 ```tf.data.Dataset.from_tensor_slices()```를 이용하여서 padding된 review data와 one-hot-encoding된 class data를 이용하여 하나로 만든다.

In [88]:
tokened_review = list(map(lambda x: x.split(' '), review))

In [89]:
len(tokened_review)

20491

In [90]:
from sklearn.model_selection import train_test_split
rev_train, rev_val, class_train, class_val = train_test_split(tokened_review, target, test_size = 0.2)
rev_train, rev_test, class_train, class_test = train_test_split(rev_train, class_train, test_size = 0.2)

In [91]:
from collections import Counter
vocabulary = Counter()
for i in range(len(tokened_review)):
    vocabulary.update(list(tokened_review[i]))

In [92]:
vocabulary.most_common()[:3]

[('hotel', 42079), ('', 40982), ('not', 30750)]

In [193]:
len(vocabulary)

102001

- vocabulary 단어 리스트에 있는 단어들중에서 모델이 알아야 하는 10000개의 단어를 넣는다
- 이렇게 자주 사용되는 단어를 vocab_size만큼 남기고 삭제해야 의미 있고 빠른 학습이 가능하다.

In [94]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size+1]
]

In [95]:
truncated_vocabulary.remove(truncated_vocabulary[1])

In [96]:
len(truncated_vocabulary)

10000

- 이제 각 단어를 ID(즉, 어휘 사전의 인덱스)로 바꾸는 전처리 단계를 추가한다.
- 1000개의 out-of-vocabulary bucket을 사용하는 lookup table을 만든다.

In [27]:
words = tf.constant(truncated_vocabulary)
words_ids = tf.range(len(truncated_vocabulary), dtype = tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, words_ids)

In [28]:
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

- 이렇게 혹시나 단어 사전에 없는 단어를 추가하게 될 oov_bucket을 추가한 vocabulary table, 즉 단어별 index를 포함한 단어 사전이 만들어졌다.
- 이를 이용해서 review의 7번째를 이용해서 index로 바꾸어 보면 아래와 같게 될 것이다.

In [29]:
table.lookup(tf.constant([review[7].split(' ')]))

<tf.Tensor: shape=(1, 87), dtype=int64, numpy=
array([[   37,   270,   502,   260,     0, 10045,     5,    53,   133,
          521,   956,  1201,    15,    49,     5,   288,   123,  1305,
          477,    15,   532,  1024,   477,  3042, 10904,   802,  1350,
           45,  4459, 10869,   802,  1576,     5, 10749,  3649, 10869,
          802,  2099,   625, 10049,  2067,     1,  6521,   395,  8590,
          546,    37,   775,   288,  1664,  2023,    38,   254,   208,
          293,  2889,    12,   886,     5,    41,    67,    53,   133,
        10715,    14,    19,    86,     6,    18,   186,    82,  1036,
         1100,    75, 10358, 10741,   436,  6082,   285,   224, 10878,
         2757,   195,    89,  1158, 10263, 10263]], dtype=int64)>

In [160]:
def encode_words(x):
    x = table.lookup(x)
    return x

In [168]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
coded_rev_train = list(map(lambda x:encode_words(tf.constant(x)), rev_train))

In [170]:
padded_rev_train = pad_sequences(coded_rev_train, padding = 'post', value = 0, truncating = 'post', maxlen = 100)

In [171]:
padded_rev_train[0]

array([ 2944,   621,     8,    13, 10416,   495,     0,  2944,  5191,
           0,   746,   152,  4289,  1557, 10567,  2560,   148,   101,
         889,   285,   165,    34,    25,   178,   303,  2023,   285,
          10,    97,   222,   147,   136,   236,  3035,   264,  1427,
          25,     0,    65,    75,    77, 10104, 10131,    64,   209,
        2117,    43,   114,     2,     7,     4,   133,  4028,   105,
          70,   139,  3340,     6,  1524,   326,   346,   382,    83,
         240,   128,   947,  5809,   402,   148,     3,  1607,    24,
         385,  1183,   913,     6,    67, 10829, 10263, 10263,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

In [186]:
coded_rev_test = list(map(lambda x:encode_words(tf.constant(x)), rev_test))
padded_rev_test = pad_sequences(coded_rev_test, padding = 'post', value = 0, truncating = 'post', maxlen = 100)
coded_rev_val = list(map(lambda x:encode_words(tf.constant(x)), rev_val))
padded_rev_val = pad_sequences(coded_rev_val, padding = 'post', value = 0, truncating = 'post', maxlen = 100)

In [196]:
def one_hot(data):
    n_classes = max(data)+1
    one_hot = np.zeros((data.shape[0],n_classes))
    one_hot[np.arange(len(data)), data] = 1
    return one_hot

In [197]:
coded_class_train = one_hot(class_train-1)
coded_class_test = one_hot(class_test-1)
coded_class_val = one_hot(class_val-1)

In [245]:
len(coded_class_train)

13113

In [199]:
BATCH_SIZE = 32
BUFFER_SIZE = 1000

In [253]:
train_set = tf.data.Dataset.from_tensor_slices((padded_rev_train, coded_class_train)).batch(BATCH_SIZE).shuffle(BUFFER_SIZE)
test_set = tf.data.Dataset.from_tensor_slices((padded_rev_test, coded_class_test)).batch(BATCH_SIZE)
val_set = tf.data.Dataset.from_tensor_slices((padded_rev_val, coded_class_val)).batch(BATCH_SIZE)

**Word-Dict - 2 Using sorted list set**
- 이렇게 하면 단어를 알파벳 순으로 정렬해서 나중에 최다 사용 단어를 사용하고 싶을 때에 어려움이 생긴다.

In [332]:
word_dict = []
tok_review = list(map(lambda x: x.split(' '), review))
for i in range(len(tok_review)):
    word_dict += sorted(list(set(tok_review[i])))

In [333]:
word_dict = list(set(word_dict))

In [334]:
len(word_dict)

102001

In [335]:
word2_idx = {}
for i in range(len(tok_review)):
    for idx, word in enumerate(tok_review[i]):
        word2_idx[word] = idx

In [336]:
review = hotel_data['Review']
tokened_review = list(map(lambda x: x.split(' '), review))
indexed_review = []
for i in range(len(tokened_review)):
    indexed_review.append(list(map(lambda x: word2_idx[x], tokened_review[i])))

In [337]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 100
padded_review = pad_sequences(indexed_review, padding = 'post', value = 0, maxlen = max_len, dtype = 'int32')

In [338]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(padded_review, target, test_size = 0.2)
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size = 0.2)

In [339]:
trainset = tf.data.Dataset.from_tensor_slices((xtrain, one_hot(ytrain-1))).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valset = tf.data.Dataset.from_tensor_slices((xval,one_hot(yval-1))).batch(BATCH_SIZE)
testset = tf.data.Dataset.from_tensor_slices((xtest, one_hot(ytest-1))).batch(BATCH_SIZE)

In [340]:
trainset.take(1)

<TakeDataset shapes: ((None, 100), (None, 5)), types: (tf.int32, tf.float64)>

### 2-2. Tokenizer + Padding

**Tokenizer**  

```keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)```

1. 이 클래스는 각 텍스트를 (딕셔너리 내 하나의 정수가 한 토큰의 색인 역할을 하는) 정수 시퀀스로, 혹은 단어 실셈이나 tf-idf 등을 기반으로 각 토큰의 계수가 이진인 벡터로 변환하여 말뭉치를 벡터화할 수 있도록 해준다.

2. Parameters
    - num_words: 단어 빈도에 따른 사용할 단어 개수의 최대값. 가장 빈번하게 사용되는 num_words개의 단어만 보존
    - filters: 문자열로, 각 성분이 텍스트에서 걸러진 문자에 해당. 디폴트 값은 모든 구두점이며, 거기에 탭과 줄 바꿈은 추가하고 ' 문자는 제외
    - lower: 불리언. 텍스트를 소문자로 변환할지 여부.
    - split: 문자열. 단어 분해 용도의 분리기.
    - char_level: 참인 경우 모든 문자가 토큰으로 처리.
    - oov_token: 값이 지정된 경우, text_to_sequence 호출 과정에서 단어색인(word_index)에 추가되어 어휘목록 외 단어를 대체.
    
3. 사용 방법
- ```tokenizer.word_index()```를 사용하면 입력된 텍스트 데이터에 사용된 모든 단어사전이 index와 함께 dictionary의 형태로 제공이 된다.
- ```tokenizer.fit_on_texts()```를 사용해서 먼저 fit를 하고
- ```tokenizer.texts_to_sequences```를 사용해서 데이터를 단어 사전에 맞추어서 index화 한다.(수치화)

In [212]:
review_data = hotel_data['Review']

In [213]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 1000, lower = True, split = ' ', char_level = False)
tokenizer.fit_on_texts(review_data)
tokened_text = tokenizer.texts_to_sequences(review_data)

In [214]:
word_dict = tokenizer.word_index

In [215]:
len(tokened_text)

20491

In [86]:
target = hotel_data['Rating']

**Padding**  
```from keras.preprocessing.sequence import pad_sequences```  
```pad_sequences(data, padding = 'post', value = 0,maxlen = max_len, dtype = 'int32')```
- 이렇게 하면 각 문장마다 설정해 준 최대 길이인 max_len만큼 뒤의 값에서 잘라주어서 만약에 길이가 부족하다면 0으로 padding을 한다.
- 이는 길이가 일정하지 않은데 scale을 맞춰주어야 하는 텍스트 데이터에서 유용하게 사용이 된다.
- 그리고 dtype는 'float32'로 바꾸어서 형태를 맞추어 주는 것도 가능하다.

In [95]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 100
text_padded = pad_sequences(tokened_text, padding = 'post', value = 0, maxlen = max_len, dtype = 'int32')

### 3. Model Making

**Embedding Layer**
```tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    **kwargs
)```  

- Turns positive integers (indexes) into dense vectors of fixed size.
1. Parameters
    - input_dim: 정수로, 총 입력할 예정인 전체 단어의 수를 의미
    - output_dim: 정수형 데이터로, embedding vector의 차원을 설정해 준다.
    - embeddings_initializer: tf.keras.initializers를 이용하는데, embedding initializer을 설정해 준다.
    - embeddings_regularizer: tf.keras.regulizers를 이용하는데, embedding regulizer을 설정해 준다.
    - embeddings_constraint: tf.keras.constraints를 이용하는데, embedding constraint를 설정해 준다.
    - mask_zero: mask_zero = True로 설정해 주면 데이터를 embedding 할 때에 padding으로 인해 0인 값을 무시한다.
    - input_length: 입력받는 하나의 sample문장의 길이  
    
2. Input shape
    : 2D tensor with shape: (batch_size, input_length).

3. Output shape
    : 3D tensor with shape: (batch_size, input_length, output_dim).

#### Counter을 이용한 데이터셋에 이용할 모델
1. Using Embedding + GRU + Dense

In [242]:
embed_size = 100
input_dim = 10000 + num_oov_buckets
input_length = 100
model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Embedding(input_dim = input_dim, output_dim = embed_size, mask_zero = True, input_length = input_length, input_shape = [None,]))
model1.add(tf.keras.layers.GRU(embed_size, return_sequences = True))
model1.add(tf.keras.layers.GRU(128))
model1.add(tf.keras.layers.Dense(5, activation = 'softmax'))

In [243]:
model1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         1100000   
_________________________________________________________________
gru_8 (GRU)                  (None, None, 100)         60600     
_________________________________________________________________
gru_9 (GRU)                  (None, 128)               88320     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 645       
Total params: 1,249,565
Trainable params: 1,249,565
Non-trainable params: 0
_________________________________________________________________


In [256]:
model1.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [257]:
val_set.take(1)

<TakeDataset shapes: ((None, 100), (None, 5)), types: (tf.int32, tf.float64)>

In [258]:
model1.fit(train_set, validation_data = val_set, epochs = 5)

Train for 410 steps, validate for 129 steps
Epoch 1/5
410/410 [==============================] - 140s 342ms/step - loss: 1.2460 - accuracy: 0.4809 - val_loss: 1.0075 - val_accuracy: 0.5299
Epoch 2/5
410/410 [==============================] - 154s 376ms/step - loss: 0.8687 - accuracy: 0.6089 - val_loss: 0.9186 - val_accuracy: 0.5911
Epoch 3/5
410/410 [==============================] - 154s 375ms/step - loss: 0.6527 - accuracy: 0.7211 - val_loss: 0.9642 - val_accuracy: 0.5879
Epoch 4/5
410/410 [==============================] - 155s 378ms/step - loss: 0.4468 - accuracy: 0.8254 - val_loss: 1.1486 - val_accuracy: 0.5860
Epoch 5/5
410/410 [==============================] - 153s 373ms/step - loss: 0.2677 - accuracy: 0.9011 - val_loss: 1.5830 - val_accuracy: 0.5704


In [259]:
model1.evaluate(test_set)

103/103 [==============================] - 4s 36ms/step - loss: 1.5743 - accuracy: 0.5587


[1.5743115625335176, 0.55870694]

2. Using Embedding + Conv1D + GRU + Dense

In [349]:
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.Embedding(input_dim = input_dim, output_dim = embed_size, input_shape = [None,], mask_zero = True, input_length = input_length))
model2.add(tf.keras.layers.Conv1D(filters = 20, kernel_size = 4, strides = 2, activation = 'relu', padding = 'valid'))
model2.add(tf.keras.layers.GRU(units = 20, return_sequences = True))
model2.add(tf.keras.layers.GRU(units = 20))
model2.add(tf.keras.layers.Dense(5, activation = 'softmax'))

In [350]:
model2.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 100)         10200100  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 20)          8020      
_________________________________________________________________
gru_12 (GRU)                 (None, None, 20)          2520      
_________________________________________________________________
gru_13 (GRU)                 (None, 20)                2520      
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 105       
Total params: 10,213,265
Trainable params: 10,213,265
Non-trainable params: 0
_________________________________________________________________


In [351]:
model2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['mae', 'accuracy'])

In [352]:
model2.fit(train_set, validation_data = val_set, epochs = 5)

Train for 410 steps, validate for 129 steps
Epoch 1/5
410/410 [==============================] - 54s 131ms/step - loss: 1.2663 - mae: 0.2646 - accuracy: 0.4558 - val_loss: 1.0692 - val_mae: 0.2409 - val_accuracy: 0.5011
Epoch 2/5
410/410 [==============================] - 54s 131ms/step - loss: 0.9395 - mae: 0.2179 - accuracy: 0.5652 - val_loss: 0.9733 - val_mae: 0.2064 - val_accuracy: 0.5579
Epoch 3/5
410/410 [==============================] - 53s 130ms/step - loss: 0.7062 - mae: 0.1689 - accuracy: 0.6944 - val_loss: 0.9942 - val_mae: 0.1988 - val_accuracy: 0.5770
Epoch 4/5
410/410 [==============================] - 53s 130ms/step - loss: 0.4823 - mae: 0.1168 - accuracy: 0.8143 - val_loss: 1.2257 - val_mae: 0.1861 - val_accuracy: 0.5714
Epoch 5/5
410/410 [==============================] - 53s 130ms/step - loss: 0.3131 - mae: 0.0744 - accuracy: 0.8943 - val_loss: 1.4951 - val_mae: 0.1877 - val_accuracy: 0.5489


In [353]:
model2.evaluate(test_set)

103/103 [==============================] - 1s 6ms/step - loss: 1.5102 - mae: 0.1880 - accuracy: 0.5505


[1.5101972442228817, 0.18804139, 0.5504727]

In [354]:
model2.evaluate(train_set)

410/410 [==============================] - 2s 6ms/step - loss: 0.2623 - mae: 0.0643 - accuracy: 0.9159


[0.26233515392352896, 0.06432104, 0.915885]

**Accuracy = 91.59%**

- 여기서 val_set와 test_set에 대해서는 예측 정확도가 그렇게 높지 않지만 train_set에 대해서는 예측값이 계속 올라가는 것으로 보아 과대적합됬다는 생각이 들었다.
    - accuracy는 계속 증가해서 89%에 가까워졌는데 validation accuracy나 test accuracy는 55.5%에 가까워졌다.

#### 직접 만든 word_dict에 적용할 model 만들기
- counter을 사용하지 않았기 때문에 무의미한 공백이나 빈도수가 높지 않은 단어만을 단어 사전에 저장했을 수 있기 때문에 학습 효과가 많이 낮을 것으로 예상한다.

In [341]:
input_dim = len(word_dict)
emb_dim = 100
input_length = 100
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Embedding(input_dim = input_dim, output_dim = emb_dim, input_length = input_length, input_shape = [None,], mask_zero = True))
model3.add(tf.keras.layers.LSTM(emb_dim, return_sequences = True))
model3.add(tf.keras.layers.LSTM(emb_dim))
model3.add(tf.keras.layers.Dense(5, activation = 'softmax'))

In [342]:
model3.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 100)         10200100  
_________________________________________________________________
lstm_10 (LSTM)               (None, None, 100)         80400     
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 505       
Total params: 10,361,405
Trainable params: 10,361,405
Non-trainable params: 0
_________________________________________________________________


In [343]:
model3.compile(loss = 'categorical_crossentropy', optimizer = 'nadam', metrics = ['accuracy'])

In [344]:
model3.fit(trainset, validation_data = valset, epochs = 5)

Train for 410 steps, validate for 103 steps
Epoch 1/5
410/410 [==============================] - 147s 358ms/step - loss: 1.2174 - accuracy: 0.4686 - val_loss: 1.2045 - val_accuracy: 0.4645
Epoch 2/5
410/410 [==============================] - 178s 434ms/step - loss: 1.1475 - accuracy: 0.4894 - val_loss: 1.1683 - val_accuracy: 0.4864
Epoch 3/5
410/410 [==============================] - 181s 441ms/step - loss: 1.1060 - accuracy: 0.5183 - val_loss: 1.1550 - val_accuracy: 0.4892
Epoch 4/5
410/410 [==============================] - 181s 441ms/step - loss: 1.0706 - accuracy: 0.5323 - val_loss: 1.1416 - val_accuracy: 0.5124
Epoch 5/5
410/410 [==============================] - 181s 442ms/step - loss: 1.0484 - accuracy: 0.5483 - val_loss: 1.1374 - val_accuracy: 0.5181


In [346]:
model3.evaluate(testset)

129/129 [==============================] - 9s 72ms/step - loss: 1.1660 - accuracy: 0.5033


[1.165988561718963, 0.5032935]

### Conclusion
- 아무래도 더 나은 학습 결과를 얻기 위해서는 Counter을 이용하여서 단어의 사용 빈도에 맞추어서 단어 사전을 만들어 index화 하는 것이 중요한 것 같다.
- 그리고 embedding layer은 자연어 처리를 할 때에 필수적으로 추가를 해야 하는데, 그 이유는 무엇보다 단어들의 의미 관계를 학습을 시켜야 하는데 그냥 index화된 데이터만 갖고는 힘들기 때문이다.
- 또한 Conv1D layer을 이용한 것이 훨씬 학습속도가 빠른데, 이는 아마도 LSTM layer이나 GRU layer이 학습을 할 때에 있어서 각 문장의 하나하나의 toeken들을 일일히 layer에 넣어서 학습 시키기 때문일 것이다.
    - RNN layer이라는 것이 결국 recursive neural network이기 때문에 이전 단계의 은닉 데이터를 이어 받아서 학습과 손실계산, 그리고 특히나LSTM의 경우 4개의 게이트를 이용해 장기 기억을 유지하면서 이전의 유의미한 데이터는 남기는 과정을 하나의 LSTM cell에서 진행하기 때문에 시간적으로 오래 걸릴 수 밖에 없다.
    - 이경우에는 many-to-one이기 때문에 결과적으로 마지막 출력값을 RNN network에서 이용해서 Dense layer에 넣어서 classification을 하는 것이므로 조금 더 수월했지만 기계 번역같이 seq-to-seq, 즉 many-to-many를 이용하는 경우에는 훨씬 더 오래 걸리고 복잡할 것이다.